In [1]:
from pipelines.analysis_pipeline import analysis_pipeline
from steps.data_loaders import get_production_df, get_train_df_evidently
from steps.prediction_steps import prediction_service_loader, predictor
from steps.drift_detection import drift_detector
from steps.data_preprocessing import preprocess_inference_data
from zenml.integrations.evidently.visualizers.evidently_visualizer import EvidentlyVisualizer
from zenml.post_execution import get_pipeline, get_pipelines
from zenml.post_execution import get_unlisted_runs

In [6]:
from zenml.integrations.evidently.steps import (
    EvidentlyProfileParameters,
    evidently_profile_step,
)

# configure the Evidently step
evidently_profile_params = EvidentlyProfileParameters(
    profile_sections=["datadrift"]
)

In [7]:
analysis_pipeline_instance = analysis_pipeline(
    inference_data_loader=get_production_df(),
    training_data_loader=get_train_df_evidently(),
    drift_detector=evidently_profile_step(
        step_name="evidently_skew_detector",
        params=evidently_profile_params,
    ),
)
analysis_pipeline_instance.run(unlisted=True)

Reloading configuration file /home/jingzhi/repos/nyc-taxi-mlops-solution/.zen/config.yaml
Creating unlisted run nyc_analysis_pipeline-19_Oct_22-09_43_07_100754 (Caching disabled)
Using stack nyc_stack to run pipeline nyc_analysis_pipeline...
Step get_production_df has started.
Step get_production_df has finished in 0.129s.
Step get_train_df_evidently has started.
Step get_train_df_evidently has finished in 0.125s.
Step evidently_skew_detector has started.
You are accessing client information from a step that has caching enabled. Future executions of this step may be cached even though the client information may be different. You should take this into consideration and adjust your step to disable caching if needed. Alternatively, use a StepContext inside your step instead, as covered here: https://docs.zenml.io/advanced-guide/pipelines/step-metadata/step-fixtures#step-contexts
You are accessing client information from a step that has caching enabled. Future executions of this step may b

In [9]:
last_run = get_unlisted_runs()[-1]

skew_detection_step = last_run.get_step(step="skew_detector")
evidently_outputs = skew_detection_step

EvidentlyVisualizer().visualize(evidently_outputs)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_4382/392719854.py:1 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4382/392719854.py'                          │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/post_execution/pipeline_run. │
│ py:68 in get_unlisted_runs                                                                       │
│                                                                                                  │
│    65 │   │   A list of post-execution run views.                                                │
│    66 │   """                                                                                    │
│    67 │   client = Client()                                                                      │
│ ❱  68 │   runs = client.zen_store.list_runs(                                                     │
│    69 │   │   project_name_or_id=client.active_project.id,                                       │
│    70 │   │   unlisted=True,                                                                     │
│    71 │   )                                                                                      │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py: │
│ 2572 in list_runs                                                                                │
│                                                                                                  │
│   2569 │   │   │   A list of all pipeline runs.                                                  │
│   2570 │   │   """                                                                               │
│   2571 │   │   if not self.runs_inside_server:                                                   │
│ ❱ 2572 │   │   │   self._sync_runs()  # Sync with MLMD                                           │
│   2573 │   │   with Session(self.engine) as session:                                             │
│   2574 │   │   │   query = select(PipelineRunSchema)                                             │
│   2575 │   │   │   if project_name_or_id is not None:                                            │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py: │
│ 3028 in _sync_runs                                                                               │
│                                                                                                  │
│   3025 │   │   │   │   │   pipeline_configuration=mlmd_run.pipeline_configuration,               │
│   3026 │   │   │   │   │   num_steps=mlmd_run.num_steps,                                         │
│   3027 │   │   │   │   )                                                                         │
│ ❱ 3028 │   │   │   │   new_run = self._create_run(new_run)                                       │
│   3029 │   │   │   │   zenml_runs[run_name] = new_run                                            │
│   3030 │   │                                                                                     │
│   3031 │   │   for run_ in zenml_runs.values():                                                  │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py: │
│ 3201 in _create_run                                        

In [11]:
analysis_run = analysis_pipeline_instance.get_runs()[-1]
drift_detection_step = analysis_run.get_step(step="drift_detector")
EvidentlyVisualizer().visualize(drift_detection_step)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_4382/3311135855.py:1 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4382/3311135855.py'                         │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/pipelines/base_pipeline.py:6 │
│ 81 in get_runs                                                                                   │
│                                                                                                  │
│   678 │   │                                                                                      │
│   679 │   │   pipeline_view = get_pipeline(cls)                                                  │
│   680 │   │   if pipeline_view:                                                                  │
│ ❱ 681 │   │   │   return pipeline_view.runs  # type: ignore[no-any-return]                       │
│   682 │   │   else:                                                                              │
│   683 │   │   │   raise RuntimeError(                                                            │
│   684 │   │   │   │   f"The PipelineView for `{cls.__name__}` could "                            │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/post_execution/pipeline.py:1 │
│ 98 in runs                                                                                       │
│                                                                                                  │
│   195 │   │   """                                                                                │
│   196 │   │   # Do not cache runs as new runs might appear during this objects                   │
│   197 │   │   # lifecycle                                                                        │
│ ❱ 198 │   │   runs = Client().zen_store.list_runs(                                               │
│   199 │   │   │   project_name_or_id=self._model.project,                                        │
│   200 │   │   │   pipeline_id=self._model.id,                                                    │
│   201 │   │   )                                                                                  │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py: │
│ 2572 in list_runs                                                                                │
│                                                                                                  │
│   2569 │   │   │   A list of all pipeline runs.                                                  │
│   2570 │   │   """                                                                               │
│   2571 │   │   if not self.runs_inside_server:                                                   │
│ ❱ 2572 │   │   │   self._sync_runs()  # Sync with MLMD                                           │
│   2573 │   │   with Session(self.engine) as session:                                             │
│   2574 │   │   │   query = select(PipelineRunSchema)                                             │
│   2575 │   │   │   if project_name_or_id is not None:                                            │
│                                                                                                  │
│ /home/jingzhi/miniconda3/envs/nyc/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py: │
│ 3028 in _sync_runs                                         